<a href="https://colab.research.google.com/github/mahmudresin/Senior-Design-Project-499/blob/main/QCNN_for_experimental_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'experimental:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5847681%2F9588433%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241014%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241014T114933Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D03345483095e21a9656840502f28b4cd1b2b7a6e14352bf3d79bdb2e538baad2fecdddc176e7bbaa110bc08095cd61181895a142a8287db3c6197d361c676226fd070cb4ea2d48deb960180d293f8124b4146b23a838241acc2b9c69e7037ffe4331a4c6efa6677431377f6a1ab11730c306e5e9c6e48c5ba52ccd580a238f266368c65877a3dbcd144144c8caad22c02d6fce74dbbbd668b17ce2bf57aac86feb26f236343f8021e6671532cd3ecf33877585b7131c1186743f33f22b4622414f6ddcfe0be4f58faa4457dc3fac3678a8db4cdde567ca3e1c6c89c6a94d52a850d530d2be3850b9f21259ee2f0def989fc718894c8e3d700f3bfd88ff259987'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy
from sklearn.model_selection import train_test_split
from pathlib import Path
from PIL import Image

# --- Check if the dataset exists ---
dataset_path = Path("/kaggle/input/experimental")  # Modified path
if not dataset_path.exists():
    raise FileNotFoundError(f"Dataset not found at: {dataset_path}")

IMAGES_PATH = Path(dataset_path / "Experimental Data" / "images")  # Modified path
MASK_PATH_TRAIN = Path(dataset_path / "Experimental Data" /"train")  # Modified path
MASK_PATH_TEST = Path(dataset_path / "Experimental Data" /"test")  # Modified path

# --- Check if required directories exist ---
required_paths = [IMAGES_PATH, MASK_PATH_TRAIN, MASK_PATH_TEST]
for path in required_paths:
    if not path.exists():
        raise FileNotFoundError(f"Directory not found: {path}")

IMG_HEIGHT, IMG_WIDTH = 256, 256  # Example image dimensions
BATCH_SIZE = 16

def load_images_and_masks(image_path, mask_path):
    images = []
    masks = []

    image_files = sorted(os.listdir(image_path))
    mask_files = sorted(os.listdir(mask_path))

    for image_file, mask_file in zip(image_files, mask_files):
        # Ensure images are opened in RGB mode or converted to RGB
        image = Image.open(os.path.join(image_path, image_file)).convert('RGB').resize((IMG_HEIGHT, IMG_WIDTH))
        mask = Image.open(os.path.join(mask_path, mask_file)).resize((IMG_HEIGHT, IMG_WIDTH))

        # Convert images to RGB format (Redundant if already opened in RGB mode)
        # image = image.convert('RGB')

        image = np.array(image) / 255.0  # Normalize images
        mask = np.array(mask) / 255.0    # Normalize masks

        images.append(image)
        masks.append(mask)

    return np.array(images), np.array(masks)

train_images, train_masks = load_images_and_masks(IMAGES_PATH, MASK_PATH_TRAIN)
test_images, test_masks = load_images_and_masks(IMAGES_PATH, MASK_PATH_TEST)